# Zeeman slower
This document provides all the information about how to design a Zeeman slower using a magnetic field.
Includes an optional interactive matplotlib widget that lets you vary key parameters with sliders.

## Theory introduction


Here, we assume we are slowing down atoms at a constant deceleration using a counter-propagating laser beam at a rate $\frac{dv}{dt} = v \frac{dv}{dz} = - a$, and when we integrate over position we get the velocity profile as a function of position. Assuming constant deceleration, we can express the velocity as a function of position z along the slower as:
$$v(z) = \sqrt{v_0^2 - 2 a z}$$
where $v_0$ is the initial velocity of the atoms at the entrance of the slower, and $a$ is the constant deceleration which typically is $a=a_{max}/2$.
Thus, the stopping distance L required to slow atoms from initial velocity $v_0$ to final velocity $v_f$ is given by:
$$L = \frac{v_0^2 - v_f^2}{2 a}$$

Assuming $v_f=0$ we can redefine $v(z)$ in terms of L as:
$$v(z) = v_0 \sqrt{1 - \frac{z}{L}}$$

When a beam of particles is slowed down using a counter-propagating laser beam, the particles experience a changing Doppler shift as they decelerate (or a chirped laser $\omega_0$). To maintain resonance between the laser frequency and the atomic transition, a spatially varying magnetic field is applied along the length of the slower.

This magnetic field induces a Zeeman shift in the atomic energy levels, compensating for the changing Doppler shift.
$$\omega_0 + \frac{\mu_B B(z)}{\hbar} = \omega_L + k v(z)$$
where $\omega_0$ is the atomic transition frequency, $\omega_L$ is the laser frequency, $\mu_B$ is the Bohr magneton, $k$ is the wavevector of the laser light, and $v(z)$ is the velocity of the atoms at position z.
$$B(z)=B_0(1-z/L)^{1/2}+B_{bias}$$
where $B_0 = \frac{\hbar k v_0}{\mu_B} = \frac{h v_0}{\lambda \mu_B}$ is the maximum magnetic field strength at the entrance of the slower, and $B_{bias}$ is a constant offset magnetic field, such that $\mu_B B_{bias} = \hbar \Delta$ where $\Delta = \omega_L - \omega_0$ is the detuning of the laser frequency from the atomic transition frequency.

In case of need to look for more spectral lines, please refer to: https://physics.nist.gov/PhysRefData/ASD/lines_form.html

Or atoms information Safranova 
https://www1.udel.edu/atom

In [27]:
%matplotlib inline

# Imports and setup
import math
import wave
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as w
from IPython.display import display, clear_output

# Physical constants (SI units)
hh = 6.62607015e-34  # J*s (Planck constant)
mu_B = 9.274009994e-24  # J/T (Bohr magneton)
amu = 1.66053906660e-27  # kg (atomic mass unit)


def compute_profile(
    L_z = .3,                    # length of the slower [m] (e.g., 0.3 m)
    wavelength=422.67276e-9,     # transition wavelength [m] (e.g., Ca-40)
    v0=500.0,                    # capture (initial) speed at z=0 [m/s]
    n_points=500                 # number of points in z grid
):
    """Compute the constant-deceleration Zeeman-slower profile.

    Inputs:
        - mass_amu: atomic mass in amu (e.g., Ca-40)
        - wavelength: transition wavelength [m] (e.g., Ca-40)
        - gamma: natural linewidth [rad/s] (Γ = 2π × FWHM in Hz)
        - g_eff: effective Landé g-factor for Zeeman shift (user

    Model and units:

    Returns:
      - z [m], v [m/s], B [T], L [m]
    """
    # Position grid and velocity profile
    zz = np.linspace(0.0, L_z, n_points)
    vv = v0 * np.sqrt(1 - zz / L_z)

    # Zeeman field profile (detuning chosen so B(L)=0)
    B_z = (hh * v0) / (mu_B * wavelength) * (1 - np.sqrt(1 - zz / L_z))

    return zz, vv, B_z, L_z


def plot_profile(z_plt, v_plt, B_plt, L_plt):
    """Plot magnetic field (in Gauss) and velocity vs position.

    - Left y-axis: B(z) in Gauss (1 T = 1e4 G).
    - Right y-axis: v(z) in m/s.
    """
    fig, ax1 = plt.subplots(figsize=(7,4))
    ax1.set_xlabel("z (m)")
    ax1.set_ylabel("B (G)", color='tab:blue')
    ax1.plot(z_plt, B_plt*1e4, color='tab:blue', lw=2, label='B(z) [G]')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.grid(True, ls='--', alpha=0.4)

    # Twin axis for velocity
    ax2 = ax1.twinx()
    ax2.set_ylabel("v (m/s)", color='tab:orange')
    ax2.plot(z_plt, v_plt, color='tab:orange', lw=2, label='v(z) [m/s]')
    ax2.tick_params(axis='y', labelcolor='tab:orange')

    ax1.set_title(f"Zeeman slower profile (L = {L_plt:.3f} m)")
    plt.tight_layout()
    plt.show()

In [28]:
# Species dropdown with mass, wavelength (m), and gamma (rad/s)
species_data = {
    'Ca-40':   {'mass': 40.078,  'wavelength': 422.67276e-9, 'gamma': 2*math.pi*34.6e6},
    'Sr-88':   {'mass': 87.62,   'wavelength': 460.862e-9,    'gamma': 2*math.pi*32.0e6},
    'Rb-87':   {'mass': 86.909,  'wavelength': 780.241e-9,    'gamma': 2*math.pi*6.065e6},
    'Cs-133':  {'mass': 132.905, 'wavelength': 852.347e-9,    'gamma': 2*math.pi*5.234e6},
    'Na-23':   {'mass': 22.990,  'wavelength': 589.158e-9,    'gamma': 2*math.pi*9.79e6},
    'Yb-174':  {'mass': 173.045, 'wavelength': 398.911e-9,    'gamma': 2*math.pi*29.0e6},
}

species = w.Dropdown(
    description='Species',
    options=list(species_data.keys()),
    value='Ca-40'
)

In [ ]:
# Sliders (continuous_update=False avoids flicker & overdraw)
v0 = w.FloatSlider(description="v0 (m/s)", value=500.0, min=0.0, max=1200.0, step=5.0, continuous_update=False)
L_z = w.FloatSlider(description="L_z (m)", value=0.3, min=0.0, max=1.0, step=0.01, continuous_update=False)
points = w.IntSlider(description="points", value=600, min=200, max=2000, step=50, continuous_update=False)

# Info display widgets
wavelength_label = w.HTML()
gamma_label = w.HTML()

# Output area
out = w.Output()


def update_info():
    d = species_data[species.value]
    wavelength_label.value = f"<b>λ</b>: {d['wavelength']*1e9:.3f} nm"
    gamma_label.value = f"<b>Γ/2π</b>: {d['gamma']/(2*math.pi)/1e6:.3f} MHz"


def refresh(change=None):
    with out:
        clear_output(wait=True)
        d = species_data[species.value]
        update_info()
        try:
            z, v, B, L = compute_profile(L_z=L_z.value, wavelength=d['wavelength'],
                                         v0=v0.value, n_points=points.value)
        except Exception as e:
            print("Parameter error:", e)
            return
        plot_profile(z, v, B, L)

for widget in (species, v0, L_z, points):
    widget.observe(refresh, names='value')

# UI container
ui = w.VBox([
    w.HBox([species, wavelength_label, gamma_label]),
    w.HBox([v0, L_z, points])
])

# Initial render
refresh()

# Display (explicit display fixes plain text repr issues)
display(ui, out)

Output()

[comment]: # (Here, we assume we are slowing down atoms at a constant deceleration using a counter-propagating laser beam at a rate $\frac{dv}{dt} = -\eta a_{max}$, where $a_{max} = \frac{\hbar k \Gamma}{2 m} \frac{s_0}{1 + s_0}$ is the maximum achievable deceleration, $\eta$ is the design parameter (0 < $\eta$ < 1), $s_0$ is the on-resonance saturation parameter, and $\Gamma$ is the natural linewidth of the atomic transition.)